In [8]:
import subprocess
print("🚀 Installing dependencies (vLLM, ngrok, fastapi)...")
subprocess.run("pip install -q vllm pyngrok uvicorn fastapi python-multipart nest-asyncio pillow", shell=True)

🚀 Installing dependencies (vLLM, ngrok, fastapi)...


CompletedProcess(args='pip install -q vllm pyngrok uvicorn fastapi python-multipart nest-asyncio pillow', returncode=0)

In [9]:
import uvicorn
import nest_asyncio
from pyngrok import ngrok
from fastapi import FastAPI, UploadFile, File
from vllm import LLM, SamplingParams
from PIL import Image
import io
import json
import traceback

# Use 7B model for A100. If using T4 (Free), switch to "Qwen/Qwen2.5-VL-3B-Instruct"
MODEL_ID = "Qwen/Qwen2.5-VL-7B-Instruct"
# MODEL_ID = "Qwen/Qwen2.5-VL-3B-Instruct" # Uncomment for Free T4 GPU

# Ngrok Auth Token (Set from user input)
ngrok.set_auth_token("38HpmQZrcDwmGKHwpgo8rFljVGY_7kgt4BNgpzhWRn8RAfjR1")

# --- 4. Initialize Model (vLLM) ---
print(f"🔄 Loading Model: {MODEL_ID}...")
try:
    # UPDATED FIX: Limit context to 8k and optimize memory
    llm = LLM(
        model=MODEL_ID,
        dtype="float16",
        gpu_memory_utilization=0.90,
        trust_remote_code=True,
        max_model_len=8192,
        limit_mm_per_prompt={"image": 1}
    )
    print("✅ Model Loaded Successfully!")
except Exception as e:
    print(f"❌ Model Load Error: {e}")
    print("If OOM on T4, try the 3B model or set gpu_memory_utilization=0.8")
    raise e

🔄 Loading Model: Qwen/Qwen2.5-VL-7B-Instruct...
INFO 01-15 09:51:58 [utils.py:253] non-default args: {'trust_remote_code': True, 'dtype': 'float16', 'max_model_len': 8192, 'disable_log_stats': True, 'limit_mm_per_prompt': {'image': 1}, 'model': 'Qwen/Qwen2.5-VL-7B-Instruct'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 01-15 09:51:59 [model.py:514] Resolved architecture: Qwen2_5_VLForConditionalGeneration
WARNING 01-15 09:51:59 [model.py:2005] Casting torch.bfloat16 to torch.float16.
INFO 01-15 09:51:59 [model.py:1661] Using max model len 8192
INFO 01-15 09:51:59 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 01-15 09:53:13 [llm.py:360] Supported tasks: ['generate']
✅ Model Loaded Successfully!


In [18]:
app = FastAPI()

@app.post("/analyze")
async def analyze_image(file: UploadFile = File(...)):
    try:
        # Read uploaded image
        content = await file.read()
        image = Image.open(io.BytesIO(content)).convert("RGB")

        # Qwen2.5-VL specific prompt format with proper placeholders
        question = """Output JSON ONLY: {"hazard": boolean, "steering": "left"|"right"|"center"|"stop", "reasoning": "short explanation"}
Where should I drive?"""

        prompt = (
            "<|im_start|>system\nYou are a robot navigator. Analyze scenes for walkability.<|im_end|>\n"
            "<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>"
            f"{question}<|im_end|>\n"
            "<|im_start|>assistant\n"
        )

        sampling_params = SamplingParams(temperature=0.1, max_tokens=128)

        # Generate with multi_modal_data
        outputs = llm.generate({
            "prompt": prompt,
            "multi_modal_data": {"image": image},
        }, sampling_params=sampling_params)

        generated_text = outputs[0].outputs[0].text
        return {"result": generated_text}

    except Exception as e:
        import traceback
        print(f"❌ ANALYZE ERROR: {e}")
        traceback.print_exc()
        return {"error": str(e), "result": json.dumps({"hazard": True, "steering": "stop", "reasoning": f"ERR: {str(e)[:40]}"})}

In [ ]:
# Open Tunnel
port = 8000
public_url = ngrok.connect(port).public_url
print(f"\nExample config.py setting:\nREMOTE_VLM_URL = \"{public_url}/analyze\"")
print(f"🚀 API Live at: {public_url}")

# Run FastAPI (Async Fix)
# uvicorn.run(app, port=port) # <--- Causes error in some Colab instances\n
config = uvicorn.Config(app, port=port)
server = uvicorn.Server(config)
await server.serve()



Example config.py setting:
REMOTE_VLM_URL = "https://informal-leona-adorably.ngrok-free.dev/analyze"
🚀 API Live at: https://informal-leona-adorably.ngrok-free.dev


INFO:     Started server process [1198]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

❌ ANALYZE ERROR: EngineCore encountered an issue. See stack trace (above) for the root cause.
INFO:     42.112.110.115:0 - "POST /analyze HTTP/1.1" 200 OK


Traceback (most recent call last):
  File "/tmp/ipython-input-1166833106.py", line 24, in analyze_image
    outputs = llm.generate({
              ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/vllm/entrypoints/llm.py", line 441, in generate
    self._validate_and_add_requests(
  File "/usr/local/lib/python3.12/dist-packages/vllm/entrypoints/llm.py", line 1630, in _validate_and_add_requests
    raise e
  File "/usr/local/lib/python3.12/dist-packages/vllm/entrypoints/llm.py", line 1617, in _validate_and_add_requests
    request_id = self._add_request(
                 ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/vllm/entrypoints/llm.py", line 1730, in _add_request
    self.llm_engine.add_request(
  File "/usr/local/lib/python3.12/dist-packages/vllm/v1/engine/llm_engine.py", line 267, in add_request
    self.engine_core.add_request(request)
  File "/usr/local/lib/python3.12/dist-packages/vllm/v1/engine/core_client.py", line 750, in add_request
    se

...

❌ ANALYZE ERROR: EngineCore encountered an issue. See stack trace (above) for the root cause.
INFO:     42.112.110.115:0 - "POST /analyze HTTP/1.1" 200 OK


Traceback (most recent call last):
  File "/tmp/ipython-input-1166833106.py", line 24, in analyze_image
    outputs = llm.generate({
              ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/vllm/entrypoints/llm.py", line 441, in generate
    self._validate_and_add_requests(
  File "/usr/local/lib/python3.12/dist-packages/vllm/entrypoints/llm.py", line 1630, in _validate_and_add_requests
    raise e
  File "/usr/local/lib/python3.12/dist-packages/vllm/entrypoints/llm.py", line 1617, in _validate_and_add_requests
    request_id = self._add_request(
                 ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/vllm/entrypoints/llm.py", line 1730, in _add_request
    self.llm_engine.add_request(
  File "/usr/local/lib/python3.12/dist-packages/vllm/v1/engine/llm_engine.py", line 267, in add_request
    self.engine_core.add_request(request)
  File "/usr/local/lib/python3.12/dist-packages/vllm/v1/engine/core_client.py", line 750, in add_request
    se

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

❌ ANALYZE ERROR: EngineCore encountered an issue. See stack trace (above) for the root cause.
INFO:     42.112.110.115:0 - "POST /analyze HTTP/1.1" 200 OK


Traceback (most recent call last):
  File "/tmp/ipython-input-1166833106.py", line 24, in analyze_image
    outputs = llm.generate({
              ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/vllm/entrypoints/llm.py", line 441, in generate
    self._validate_and_add_requests(
  File "/usr/local/lib/python3.12/dist-packages/vllm/entrypoints/llm.py", line 1630, in _validate_and_add_requests
    raise e
  File "/usr/local/lib/python3.12/dist-packages/vllm/entrypoints/llm.py", line 1617, in _validate_and_add_requests
    request_id = self._add_request(
                 ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/vllm/entrypoints/llm.py", line 1730, in _add_request
    self.llm_engine.add_request(
  File "/usr/local/lib/python3.12/dist-packages/vllm/v1/engine/llm_engine.py", line 267, in add_request
    self.engine_core.add_request(request)
  File "/usr/local/lib/python3.12/dist-packages/vllm/v1/engine/core_client.py", line 750, in add_request
    se